In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

for dirname, _, filenames in os.walk("/kaggle/input"):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"

# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
file_train_data = "data/train.csv"
file_test_data = "data/test.csv"

In [ ]:
import csv


train_csv = []
with open(file_train_data, "r") as f:
    reader = csv.DictReader(f)

    for row in reader:
        train_csv.append(row)



test_csv = []
with open(file_test_data, "r") as f:
    reader = csv.DictReader(f)

    for row in reader:
        test_csv.append(row)

print(f"Train data size: {len(train_csv)}")
print(train_csv[:3])
print(f"Columns ({len(train_csv[0].keys())}):", train_csv[0].keys())

print(f"Test data size: {len(test_csv)}")
print(test_csv[:3])
print(f"Columns: ({len(test_csv[0].keys())})", test_csv[0].keys())

Train data size: 1460
[{'Id': '1', 'MSSubClass': '60', 'MSZoning': 'RL', 'LotFrontage': '65', 'LotArea': '8450', 'Street': 'Pave', 'Alley': 'NA', 'LotShape': 'Reg', 'LandContour': 'Lvl', 'Utilities': 'AllPub', 'LotConfig': 'Inside', 'LandSlope': 'Gtl', 'Neighborhood': 'CollgCr', 'Condition1': 'Norm', 'Condition2': 'Norm', 'BldgType': '1Fam', 'HouseStyle': '2Story', 'OverallQual': '7', 'OverallCond': '5', 'YearBuilt': '2003', 'YearRemodAdd': '2003', 'RoofStyle': 'Gable', 'RoofMatl': 'CompShg', 'Exterior1st': 'VinylSd', 'Exterior2nd': 'VinylSd', 'MasVnrType': 'BrkFace', 'MasVnrArea': '196', 'ExterQual': 'Gd', 'ExterCond': 'TA', 'Foundation': 'PConc', 'BsmtQual': 'Gd', 'BsmtCond': 'TA', 'BsmtExposure': 'No', 'BsmtFinType1': 'GLQ', 'BsmtFinSF1': '706', 'BsmtFinType2': 'Unf', 'BsmtFinSF2': '0', 'BsmtUnfSF': '150', 'TotalBsmtSF': '856', 'Heating': 'GasA', 'HeatingQC': 'Ex', 'CentralAir': 'Y', 'Electrical': 'SBrkr', '1stFlrSF': '856', '2ndFlrSF': '854', 'LowQualFinSF': '0', 'GrLivArea': '1710

In [3]:
import copy
import itertools

remove_columns = ["MoSold", "YrSold"]
prediction_columns = ["SalePrice"]

def parse_data(lines):
    # First we filter out the data we need
    data = []
    for l in lines:
        data.append(l)
        # print(l)

    # Remove columns we don't need
    for r in data:
        for col in remove_columns:
            if col in r:
                del r[col]
    
    data_type_analysis = {}

    # Check each column for a best guess data type
    print("Guess data types")
    print("================")
    for k in data[0].keys():
        if k == "Id":
            continue        
        print(f"Key: {k}")
        values = [d[k] for d in data]
        unique_values = set(values)
        print(f"  Unique values ({len(unique_values)}): {unique_values}")
        if all(v.isdigit() or (v.startswith('-') and v[1:].isdigit() or (v == 'NA') or (v == '')) for v in unique_values):
            print("  Data type: Number")
            data_type_analysis[k] = "Number"
        else:
            print("  Data type: Categorical")
            data_type_analysis[k] = "Categorical"

    # Create mappings for categorical data
    normalized_categorial_mappings = {}
    print("Categorial mappings")
    print("===================")
    for k in data_type_analysis.keys():
        print(f"Key: {k}, Type: {data_type_analysis[k]}")
        if data_type_analysis[k] == "Categorical":
            unique_values = list(sorted(set(d[k] for d in data)))
            print(f"  Categorical values: {unique_values}")
            mapping = {v: i for i, v in enumerate(unique_values)}
            normalized_categorial_mappings[k] = mapping
            print(f"  Mapping: {mapping}")
        
    # Transform data according to type
    print("Transform data according to type")
    print("================================")    
    for l in data:
        for k in l.keys():
            if k == "Id":
                continue

            if data_type_analysis[k] == "Number":
                if l[k] == 'NA' or l[k] == '':
                    l[k] = 0.0
                else:
                    l[k] = float(l[k])
            else:
                # Set categorical values as integers
                l[k] = normalized_categorial_mappings[k][l[k]]

    # Normalize columns

    ####################################################################################

    normalized_data = copy.deepcopy(data)
    keys = list(normalized_data[0].keys())

    # Extract min max of each key
    keys_minmax = {}
    for k in keys:
        values = [d[k] for d in normalized_data if k in d]
        # print(values)
        min_value, max_value = float(min(values)), float(max(values))
        keys_minmax[k] = {"min": min_value, "max": max_value}

    # Normalize 0 to 1 all keys
    for r in normalized_data:
        for k in keys:
            if k == "id":
                continue
            value = float(r[k])
            minmax = keys_minmax[k]
            norm_value = (value - minmax["min"]) / (minmax["max"] - minmax["min"]) if (minmax["max"] - minmax["min"]) != 0 else 0.0
            r[k] = norm_value

    # Split data into input -> expected output
    train_data = []
    for r in normalized_data:
        train_data.append(
            {
                "id": r["Id"],
                "input": list(
                    {k: v for k, v in r.items() if k not in list(itertools.chain(*[prediction_columns, ["Id"]]))}.values()
                ),
                "output": [r.get(c, 0.0) for c in prediction_columns],
            }
        )

    return train_data

train_data = parse_data(train_csv)
print(train_data[:3])

test_data = parse_data(test_csv)
print(test_data[:3])

# TODO: extract normalized mappings from both datasets and provide them when parsing data.

Guess data types
Key: MSSubClass
  Unique values (15): {'160', '85', '40', '80', '70', '60', '120', '50', '30', '45', '90', '180', '75', '20', '190'}
  Data type: Number
Key: MSZoning
  Unique values (5): {'RH', 'FV', 'RL', 'RM', 'C (all)'}
  Data type: Categorical
Key: LotFrontage
  Unique values (111): {'36', '88', '48', '77', '62', '83', '134', '73', '90', '45', '112', '130', '57', '144', '137', '105', '87', 'NA', '153', '95', '102', '68', '115', '63', '94', '118', '55', '71', '46', '34', '81', '128', '109', '43', '74', '110', '69', '93', '120', '97', '78', '50', '40', '44', '51', '67', '76', '56', '38', '121', '92', '84', '24', '39', '91', '58', '21', '66', '174', '37', '82', '106', '41', '86', '96', '85', '114', '70', '129', '79', '160', '116', '152', '33', '150', '182', '89', '72', '168', '52', '49', '141', '99', '60', '32', '111', '103', '98', '108', '101', '30', '122', '313', '75', '35', '54', '42', '140', '61', '53', '80', '65', '149', '107', '124', '104', '47', '64', '100', '

In [4]:
import torch.nn as nn
import torch
import numpy as np
from typing import List

# Determine the best available device
def get_device():
    if torch.cuda.is_available():
        return torch.device("cuda")
    elif torch.backends.mps.is_available():
        return torch.device("mps")
    else:
        return torch.device("cpu")


# DEVICE = torch.device("cpu")
DEVICE = get_device()
print(DEVICE)

class SimpleNeuralNetwork(nn.Module):
    """Simple feedforward neural network using PyTorch"""

    def __init__(
        self,
        input_size: int = 16,
        hidden_layers: List[int] = [256],
        output_size: int = 4,
        empty: bool = False,
    ):
        super().__init__()

        if empty:
            return

        self.input_size = input_size
        self.hidden_layers = hidden_layers
        self.output_size = output_size

        # Build layers using PyTorch modules
        layers = []
        prev_size = input_size

        # Add hidden layers
        for hidden_size in hidden_layers:
            layers.append(nn.Linear(prev_size, hidden_size))
            layers.append(nn.ReLU())
            # layers.append(nn.Sigmoid())
            prev_size = hidden_size

        # Add output layer (no activation)
        layers.append(nn.Linear(prev_size, output_size))

        self.network = nn.Sequential(*layers)

        # Initialize weights using He initialization
        self._initialize_weights()

        # Move to device
        self.to(DEVICE)

    def _initialize_weights(self):
        """Initialize weights using He initialization"""
        for module in self.modules():
            if isinstance(module, nn.Linear):
                nn.init.kaiming_normal_(module.weight, nonlinearity="tanh")
                nn.init.zeros_(module.bias)

    def forward(self, x):
        """Forward pass through the network"""
        # Convert numpy array to tensor if needed and move to device
        if isinstance(x, np.ndarray):
            x = torch.from_numpy(x).float().to(DEVICE)
        elif isinstance(x, torch.Tensor):
            x = x.to(DEVICE)
        elif isinstance(x, list):
            x = torch.tensor(x, dtype=torch.float32).to(DEVICE)

        return self.network(x)

    def mutate(self, mutation_rate: float = 0.1, mutation_strength: float = 0.5):
        """Mutate the network's weights and biases"""
        with torch.no_grad():
            for param in self.parameters():
                if torch.rand(1).item() < mutation_rate:
                    mutation = torch.randn_like(param) * mutation_strength
                    param.add_(mutation)

cuda


In [5]:
from joblib import Parallel, delayed
import joblib
from typing import List
from typing import Tuple
import numpy as np
import random


class EvolutionaryOptimizer:
    def __init__(
        self,
        population_size: int = 50,
        elite_size: int = 10,
        new_members: int = 0,
        mutation_rate: float = 0.1,
        mutation_strength: float = 0.5,
        input_size: int = 8,
        hidden_layers: List[int] = [32],
        output_size: int = 1,
        train_data: List = [],
        train_size: int = 0,
    ):
        self.population_size = population_size
        self.elite_size = elite_size
        self.new_members = new_members
        self.mutation_rate = mutation_rate
        self.mutation_strength = mutation_strength
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_layers = hidden_layers
        self.train_data = train_data
        self.train_size = train_size if train_size > 0 else len(train_data)

        # Convert training data to tensors once
        self.input_tensor = torch.tensor([data['input'] for data in train_data], dtype=torch.float32).to(DEVICE)
        self.output_tensor = torch.tensor([data['output'] for data in train_data], dtype=torch.float32).to(DEVICE)

        # Create initial population
        self.population = []
        for _ in range(population_size):
            network = SimpleNeuralNetwork(
                input_size=input_size,
                output_size=output_size,
                hidden_layers=hidden_layers,
            )
            self.population.append(network)

    def evaluate(
        self,
    ) -> List[Tuple[SimpleNeuralNetwork, int, float]]:
        if DEVICE.type == "cuda" or DEVICE.type == "mps":
            # GPU evaluation - process all networks sequentially since GPU handles parallelization
            results = []
            for network in self.population:
                network.eval()
                with torch.no_grad():
                    predictions = network(self.input_tensor)
                    predictions = (predictions > 0.5).float()
                    accuracy = (predictions == self.output_tensor).float().mean().item()
                    results.append((network, accuracy))
            return results
        else:
            # CPU evaluation - use joblib for parallel processing
            def eval_network(network: SimpleNeuralNetwork):
                network.eval()
                scores = []
                with torch.no_grad():
                    for test in self.train_data:
                        input_values = test['input']
                        prediction = network.forward(input_values)
                        prediction = True if prediction[0] > .5 else False
                        reality = True if test['output'][0] == 1.0 else False
                        scores.append(1 if prediction == reality else 0)
                return (network, sum(scores) / len(scores))

            results = Parallel(n_jobs=joblib.cpu_count())(
                delayed(eval_network)(net) for net in self.population
            )
            return results

    def select_and_breed(
        self, evaluated: List[Tuple[SimpleNeuralNetwork, int, float]]
    ) -> None:
        # Sort by score descending
        evaluated.sort(key=lambda x: x[1], reverse=True)
        elite = evaluated[: self.elite_size]

        new_population = []
        # Keep elite networks
        for net, _ in elite:
            new_population.append(net)

        # Create offspring by mutating elite networks
        while len(new_population) < self.population_size:
            parent = random.choice(elite)[0]

            # Create a child by copying the parent's state
            child = copy.deepcopy(parent)

            # Mutate the child
            child.mutate(self.mutation_rate, self.mutation_strength)
            new_population.append(child)

        # Add random new members
        for _ in range(self.new_members):
            network = SimpleNeuralNetwork(hidden_layers=self.hidden_layers, input_size=self.input_size, output_size=self.output_size)
            new_population.append(network)

        self.population = new_population

    def run_generation(
        self,
    ) -> Tuple[List[SimpleNeuralNetwork], float]:
        evaluated = self.evaluate()
        best_precision = max(precision for _, precision in evaluated)

        self.select_and_breed(evaluated)

        return self.population, best_precision

In [6]:
import pickle

def save_network(network: SimpleNeuralNetwork, filename: str):
    torch.save(network.state_dict(), filename)


def load_network(filename: str, hidden_layers: List[int]) -> SimpleNeuralNetwork:
    network = SimpleNeuralNetwork(hidden_layers=hidden_layers)
    network.load_state_dict(torch.load(filename, map_location=DEVICE))
    network.to(DEVICE)
    return network


def save_population(population: List[SimpleNeuralNetwork], filename: str):
    with open(filename, "wb") as f:
        pickle.dump(population, f)


def load_population(filename: str) -> List[SimpleNeuralNetwork]:
    with open(filename, "rb") as f:
        population = pickle.load(f)
    return population

In [7]:
import time
from datetime import timedelta

generations = 1000
hidden_layers = [16, 8, 4, 2]
mutation_rate = 0.25
mutation_strength = 0.25

folder = f"networks/{'_'.join(str(x) for x in hidden_layers)}"
os.makedirs(folder, exist_ok=True)
print(f"Saving networks to folder: {folder}")

scores_best = []

def mutation_decay(x, mu0=0.25, k=3.0):
    return mu0 * np.exp(-k * x)

start_time = time.time()

epoch = 0
last_precision = 0
while True:
    print(f"Epoch {epoch+1} starting...")
    epoch += 1

    optimizer = EvolutionaryOptimizer(
        population_size=100,
        elite_size=10,
        new_members=0,
        mutation_rate=mutation_rate,
        mutation_strength=mutation_strength,
        input_size=len(train_data[0]['input']),
        hidden_layers=hidden_layers,
        output_size=1,
        train_data=random.sample(train_data, len(train_data) // 4),
    )

    for gen in range(generations):
        optimizer.mutation_strength = mutation_decay(
            gen / generations, k=8, mu0=mutation_strength
        )
        optimizer.mutation_rate = mutation_decay(gen / generations, k=8, mu0=mutation_rate)

        (population, best_precision) = optimizer.run_generation()
        # print(f"Epoch: {epoch} {gen+1}/{generations} - Best Precision: {best_precision:.4f}")

    elapsed_time = time.time() - start_time
    duration = str(timedelta(seconds=(elapsed_time)))
    print(f"Epoch {epoch} completed after {duration}")
    if best_precision > last_precision:
        save_population(optimizer.population, f"{folder}/population_epoch_{epoch}.pkl")
        last_precision = best_precision    
        print(f"Saved population for epoch {epoch} with precision {best_precision:.4f}")

Saving networks to folder: networks/16_8_4_2
Epoch 1 starting...
Epoch 1 completed after 0:01:28.458615
Saved population for epoch 1 with precision 0.0027
Epoch 2 starting...
Epoch 2 completed after 0:02:56.750283
Epoch 3 starting...


KeyboardInterrupt: 

In [8]:
# Generate submission files for all saved populations
for dirname, _, filenames in os.walk(folder):
    for filename in filenames:
        print(os.path.join(dirname, filename))

        population = load_population(os.path.join(dirname, filename))
        network = population[0]

        checkpoint = filename.split("_")[-1].split(".")[0]
        print(f"Generating submission for checkpoint {checkpoint}...")

        predictions = []
        for test in test_data:
            input_values = test['input']
            prediction = network.forward(input_values)
            # prediction = prediction.cpu()
            prediction = True if prediction[0] > .5 else False
            # print(f"Passenger ID: {test['id']}, Survived: {1 if prediction else 0}")
            predictions.append([test['id'], 1 if prediction else 0])

        # Save predictions to CSV
        with open(f"submission_{checkpoint}.csv", "w", newline="") as csvfile:
            fieldnames = ["PassengerId", "Survived"]
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

            writer.writeheader()
            for pid, survived in predictions:
                writer.writerow({"PassengerId": pid, "Survived": survived})



# population = load_population('networks/256/population_epoch_228.pkl')
# network = population[0]

networks/16_8_4_2/population_epoch_1.pkl
Generating submission for checkpoint 1...


In [ ]:
# Select network from the optimizer
network = optimizer.population[0]

In [ ]:
predictions = []
for test in test_data:
    input_values = test['input']
    prediction = network.forward(input_values)
    # prediction = prediction.cpu()
    prediction = True if prediction[0] > .5 else False
    # print(f"Passenger ID: {test['id']}, Survived: {1 if prediction else 0}")
    predictions.append([test['id'], 1 if prediction else 0])

# Save predictions to CSV
with open("submission.csv", "w", newline="") as csvfile:
    fieldnames = ["PassengerId", "Survived"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for pid, survived in predictions:
        writer.writerow({"PassengerId": pid, "Survived": survived})


In [ ]:
mutation_decay(1, mu0=0.25, k=8.0)

np.float64(8.386565697562796e-05)